In [1]:
import os
import getpass

from dotenv import load_dotenv
load_dotenv("../keys.env")

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.chat_models import init_chat_model

## 일반적인 LLM 로드 방법
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


## 프롬프트 템플릿 정의
## 주어진 텍스트에서 원하는 정보를 추출해라. 
## "Classification" function에서 언급된 속성만 추출한다.
tagging_prompt = ChatPromptTemplate.from_template("""
Extract the desired information from the following passage.
Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

In [4]:
## 스키마 모델.
## LLM이 스키마에 정의된 항목과 값을 추론하고 결과로 반환한다.

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text") ## 긍정, 부정 감정 : 긍정, 부정, 중립
    aggressiveness: int = Field(description="How aggressive the text is on a scale from 1 to 10") ## 공격성 측정 : 1~10
    language: str = Field(description="The language the text is written in") ## 사용된 언어

In [5]:
## 스키마를 반영하여 LLM이 정해진 출력을 반환하도록 만든다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

In [6]:
inp = "당신을 만나서 정말 기분이 나쁘다. 우리는 두 번 다시 만나서는 안될 것 같다!!!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='negative', aggressiveness=8, language='Korean')

In [8]:
import openai

In [9]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

functions = [
    {
        "name": "classify_text",
        "description": "Classify the text into sentiment, aggressiveness and language.",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "string",
                    "enum": ["happy", "neutral", "sad"]
                },
                "aggressiveness": {
                    "type": "integer",
                    "enum": [1, 2, 3, 4, 5],
                    "description": "Higher value means more aggressive"
                },
                "language": {
                    "type": "string",
                    "enum": ["spanish", "english", "french", "german", "italian"]
                }
            },
            "required": ["sentiment", "aggressiveness", "language"]
        }
    }
]

response = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {"role": "system", "content": "You are a helpful classifier."},
        {"role": "user", "content": "Estoy muy enojado con vos! Te voy a dar tu merecido!"}
    ],
    functions=functions,
    function_call={"name": "classify_text"}
)

print(response.choices[0].message.function_call.arguments)


{"sentiment":"sad","aggressiveness":5,"language":"spanish"}


In [10]:
import os
import json
import openai

## OpenAI API 키 설정
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 두 개의 function 정의
## 첫 번째는 텍스트를 분류하는 함수
## 두 번째는 텍스트를 요약하는 함수
functions = [
    {
        "name": "classify_text",
        "description": "Classify the text into sentiment, aggressiveness and language.",
        "parameters": {
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "string",
                    "enum": ["happy", "neutral", "sad"]
                },
                "aggressiveness": {
                    "type": "integer",
                    "enum": [1, 2, 3, 4, 5],
                    "description": "Higher value means more aggressive"
                },
                "language": {
                    "type": "string",
                    "enum": ["spanish", "english", "french", "german", "italian"]
                }
            },
            "required": ["sentiment", "aggressiveness", "language"]
        }
    },
    {
        "name": "summarize_text",
        "description": "Summarize the input text into one short sentence.",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Short summary of the input text."
                }
            },
            "required": ["summary"]
        }
    }
]

In [11]:
# 테스트할 입력 문장
user_input = "Estoy muy enojado con vos! Te voy a dar tu merecido!"

# Chat 호출 (function_call을 명시하지 않음 = GPT가 선택)
response = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_input}
    ],
    functions=functions
)

# 결과 출력
message = response.choices[0].message

print("\n✅ 선택된 함수 이름:")
print(message.function_call.name)

print("\n📦 반환된 arguments:")
print(json.loads(message.function_call.arguments))



✅ 선택된 함수 이름:
classify_text

📦 반환된 arguments:
{'sentiment': 'sad', 'aggressiveness': 5, 'language': 'spanish'}
